### ESCOM - IIA
#### FUNDAMENTOS DE INTELIGENCIA ARTIFICIAL - PROYECTO ML
#### Semestre 2023-2 - Grupo 4BV2
--- 
##### Autores:
- **Valdés Luis Eliot Fabián**

In [ ]:
# importamos la librerias necesarias
from ucimlrepo import fetch_ucirepo  # libreria de repositorios de datasets
import pandas as pd  # libreria para el manejo de dataframes

----
# PASO 1: Carga de datos
Obtener el dataset, describirlo de manera general usando estadisticas

In [ ]:
# definimos los datasets a utilizar
datasets = {
    "Eliot": 53, # Iris
    "Ethel": 728, # 
    "Leo": 878, # Cirrhosis
    "Adair": 109, # Wine
}
dataset = fetch_ucirepo(id=datasets["Eliot"]) # cargamos el dataset que deseemos utilizar

In [ ]:
# mostramos un pequeño resumen de lo que trata el dataset 
dataset.metadata.abstract

In [ ]:
# obtenemos los datos
X = dataset.data.features 
y = dataset.data.targets 

In [ ]:
# creamos un dataframe con los datos
df = pd.DataFrame(X, columns=dataset.data.feature_names)
df['target'] = y

# mostramos la cantidad de registros
print(f'Cantidad de registros: {len(df)}')

# imprimimos los primeros 5 registros
print(df.head())

In [ ]:
# mostramos los tipos de datos de cada columna
print(f"Tipos de datos (Metodo pandas):\n{df.dtypes}\n")

In [ ]:
# mostramos los tipos de datos usando los metodos de la librearia de repositorios
dataframe_tipos = pd.DataFrame({'Variable Name': dataset.variables['name'], 'Type': dataset.variables['type']})
print(f"Tipos de datos (Metodo ucimlrepo):\n{dataframe_tipos}\n")

In [ ]:
# por cada columna hacer lo siguiente 
# si el tipo de dato es numerico, mostrar la media, mediana, desviacion estandar, minimo y maximo
# si el tipo de dato es categorico, mostrar la cantidad de valores unicos

# limitamos la cantidad de columnas a 10
for col in df.columns[:10]:
    if df[col].dtype == 'float64' or df[col].dtype == 'int64':
        print(f"=> '{col}':")
        print(f"\t-Media: {df[col].mean()}")
        print(f"\t-Mediana: {df[col].median()}")
        print(f"\t-Desviacion estandar: {df[col].std()}")
        print(f"\t-Minimo: {df[col].min()}")
        print(f"\t-Maximo: {df[col].max()}")
    else:
        print(f"=> '{col}':")
        print(f"\t-Valores unicos: {df[col].unique()}")
        print(f"\t-Cantidad de valores unicos: {df[col].nunique()}")

----

# PASO 2: PREPROCESAMIENTO DE DATOS

Separamos el dataset en los vectores de entrada X & de salida Y(las clases que son definidas con tipo de dato categorico)


In [ ]:
# el vector de entrada X son todas las columnas menos la columna target
vector_x = df.drop(['target'], axis=1)
# mostramos los primeros 5 registros del vector de entrada X
print(vector_x.head())

In [ ]:
# el vector de salida Y es la columna target
vector_y = df['target']
# mostramos los primeros 5 registros del vector de salida Y
print(vector_y.head())

In [ ]:
# creamos una funcion que recibe como parametros el vector de entrada X y el vector de salida Y
def describe_categories(vector_x, vector_y):
    # agrupamos el vector de entrada X de acuerdo al vector de salida Y
    grouped = vector_x.groupby(vector_y)
    for name, group in grouped:
        print("\n",("="*50))
        print(f'Clase: {name}')
        print("="*50)
        # por cada grupo, accedemos a cada una de las columnas
        for col in group.columns:
            # validamos que el tipo de dato de la columna sea numerico y de ser el caso mosstramos las estadisticas
            if group[col].dtype == 'float64' or group[col].dtype == 'int64':
                print(f"=> '{col}':")
                print(f"\t-Media: {group[col].mean()}")
                print(f"\t-Mediana: {group[col].median()}")
                print(f"\t-Desviacion estandar: {group[col].std()}")
                print(f"\t-Minimo: {group[col].min()}")
                print(f"\t-Maximo: {group[col].max()}")
            else:
                print(f"=> '{col}':")
                print(f"\tValores unicos: {group[col].unique()}")
                print(f"\tCantidad de valores unicos: {group[col].nunique()}")            
        

describe_categories(vector_x, vector_y)